In [68]:
import os
import re
import numpy as np
import mne
from mne.io import read_raw_brainvision

def get_rawdata_list(): # get the list of rawdata files with .vhdr extension
    rawdata_dir = 'C:/Users/YSB/Desktop/Data/2022_sternberg_tACS/' # directory where the rawdata is stored
    rawdata_list_total = os.listdir(rawdata_dir) # return all files in the directory without directory itself
    rawdata_list = [os.path.join(rawdata_dir, i) for i in rawdata_list_total if re.search('[0-9].vhdr', i)] # return only files with .vhdr extension
    
    return rawdata_list

def get_montage():
    montage_dir = 'C:/Users/YSB/Desktop/Data/2022_sternberg_tACS/CMA-64_REF.bvef'
    montage = mne.channels.read_custom_montage(montage_dir)
    montage.rename_channels(dict(REF='FCz'))
    
    return montage

def basic_filtering(raw):
    l_freq = 0.5 # low cut-off frequency
    h_freq = None
    iir_params = mne.filter.construct_iir_filter( 
        {'ftype': 'butter', 'order': 4},
        l_freq, None, raw.info['sfreq'],
        'highpass', return_copy=False, verbose=None
    ) # IIR filter parameters
    raw_filtered = raw.copy().filter(
        l_freq=l_freq, h_freq = h_freq, method='iir', 
        iir_params=iir_params, verbose=None
    ) # bandpass filtering
    raw_filtered = raw_filtered.notch_filter(60) # notch filtering
    
    return raw_filtered

def get_epochs(raw):
    events, _ = mne.events_from_annotations(raw)
    
    events_of_interest = mne.pick_events(events, include = [61, 62])
    epochs = mne.Epochs(raw, events_of_interest, tmin=0 + 0.033, tmax = (9000-2)/raw.info['sfreq'] + 0.033, baseline = (None, None), preload=True)
    
    return epochs, events

In [69]:

rawdata_list = get_rawdata_list()
montage = get_montage()
raw = read_raw_brainvision(rawdata_list[0], preload=True)
raw.set_channel_types({'EOG':'eog'})
raw.set_montage(montage)
raw_filtered = basic_filtering(raw)

Extracting parameters from C:/Users/YSB/Desktop/Data/2022_sternberg_tACS/tACS_2022_April_SU0001_1.vhdr...
Setting channel info structure...
Reading 0 ... 276699  =      0.000 ...   553.398 secs...



IIR filter parameters
---------------------
Butterworth highpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 8 (effective, after forward-backward)
- Cutoff at 0.50 Hz: -6.02 dB

Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 0.5 Hz



C:\Users\YSB\AppData\Local\Temp\ipykernel_21348\2148696495.py:5: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  raw.set_montage(montage)


Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)
- Upper passband edge: 60.65 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 60.90 Hz)
- Filter length: 3301 samples (6.602 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:    0.9s finished


In [85]:
fig = raw.plot(n_channels=64, duration=240.0, scalings={'eeg':100e-6})

Channels marked as bad:
none


In [84]:
raw.interpolate_bads(reset_bads = True, verbose = None)

Interpolating bad channels
    Automatic origin fit: head of radius 95.0 mm
Computing interpolation matrix from 61 sensor positions
Interpolating 2 sensors


<RawBrainVision | tACS_2022_April_SU0001_1.eeg, 64 x 276700 (553.4 s), ~135.2 MB, data loaded>

In [130]:
events

array([[     0,      0,  99999],
       [     0,      0,  10001],
       [  4151,      0,     61],
       [  7142,      0,     71],
       [ 11642,      0,     81],
       [ 12159,      0,     91],
       [ 12642,      0,     41],
       [ 13142,      0,     62],
       [ 16142,      0,     72],
       [ 20642,      0,     82],
       [ 20911,      0,     92],
       [ 21642,      0,     42],
       [ 22142,      0,     61],
       [ 25142,      0,     71],
       [ 29642,      0,     81],
       [ 30015,      0,     91],
       [ 30642,      0,     41],
       [ 31142,      0,     62],
       [ 34142,      0,     72],
       [ 38642,      0,     82],
       [ 39039,      0,     92],
       [ 39642,      0,     42],
       [ 40142,      0,     62],
       [ 43142,      0,     72],
       [ 47642,      0,     82],
       [ 48059,      0,     94],
       [ 48642,      0,     42],
       [ 49142,      0,     61],
       [ 52141,      0,     71],
       [ 56641,      0,     81],
       [ 5

In [195]:
events[(events[:, 2]!=71) & (events[:, 2]!=72) & (events[:, 0]!=0), :]+=delaying_sample_matrix

ValueError: operands could not be broadcast together with shapes (120,3) (152,6) (120,3) 

In [205]:
np.add.at(events, (events[:, 2]!=71) & (events[:, 2]!=72) & (events[:, 0]!=0), delaying_sample)

In [206]:
events

array([[     0,      0,  99999],
       [     0,      0,  10001],
       [  4167,     16,     77],
       [  7142,      0,     71],
       [ 11658,     16,     97],
       [ 12175,     16,    107],
       [ 12658,     16,     57],
       [ 13158,     16,     78],
       [ 16142,      0,     72],
       [ 20658,     16,     98],
       [ 20927,     16,    108],
       [ 21658,     16,     58],
       [ 22158,     16,     77],
       [ 25142,      0,     71],
       [ 29658,     16,     97],
       [ 30031,     16,    107],
       [ 30658,     16,     57],
       [ 31158,     16,     78],
       [ 34142,      0,     72],
       [ 38658,     16,     98],
       [ 39055,     16,    108],
       [ 39658,     16,     58],
       [ 40158,     16,     78],
       [ 43142,      0,     72],
       [ 47658,     16,     98],
       [ 48075,     16,    110],
       [ 48658,     16,     58],
       [ 49158,     16,     77],
       [ 52141,      0,     71],
       [ 56657,     16,     97],
       [ 5

In [194]:
a = np.ones(np.shape(events)[0]).reshape(-1, 1)
b = np.zeros((np.shape(events)[0], 2))
delaying_sample_matrix = np.concatenate((a, b), axis=1)
delaying_sample_matrix.dtype = int


In [132]:
# modules
import pickle
import math
# params
srate = raw.info['sfreq']
delaying_sample = math.floor(srate/1000*33)
tmin = -300/srate
tmax = (9300-1)/srate

events, _ = mne.events_from_annotations(raw)
events[(events[:, 2]!=71) & (events[:, 2]!=72) & (events[:, 0]!=0), :][:, 0]+=delaying_sample
events_of_interest = mne.pick_events(events, include = [61, 62])
epochs = mne.Epochs(raw = raw, events = events_of_interest, 
                    tmin = tmin, tmax = tmax, baseline = None,
                    reject = None, preload=True, verbose = None)
events

Used Annotations descriptions: ['Comment/actiCAP Data On', 'New Segment/', 'Stimulus/S 41', 'Stimulus/S 42', 'Stimulus/S 61', 'Stimulus/S 62', 'Stimulus/S 71', 'Stimulus/S 72', 'Stimulus/S 81', 'Stimulus/S 82', 'Stimulus/S 91', 'Stimulus/S 92', 'Stimulus/S 93', 'Stimulus/S 94']
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 30 events and 9600 original time points ...


0 bad epochs dropped


array([[     0,      0,  99999],
       [     0,      0,  10001],
       [  4151,      0,     61],
       [  7142,      0,     71],
       [ 11642,      0,     81],
       [ 12159,      0,     91],
       [ 12642,      0,     41],
       [ 13142,      0,     62],
       [ 16142,      0,     72],
       [ 20642,      0,     82],
       [ 20911,      0,     92],
       [ 21642,      0,     42],
       [ 22142,      0,     61],
       [ 25142,      0,     71],
       [ 29642,      0,     81],
       [ 30015,      0,     91],
       [ 30642,      0,     41],
       [ 31142,      0,     62],
       [ 34142,      0,     72],
       [ 38642,      0,     82],
       [ 39039,      0,     92],
       [ 39642,      0,     42],
       [ 40142,      0,     62],
       [ 43142,      0,     72],
       [ 47642,      0,     82],
       [ 48059,      0,     94],
       [ 48642,      0,     42],
       [ 49142,      0,     61],
       [ 52141,      0,     71],
       [ 56641,      0,     81],
       [ 5